Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [ ]:
url = 'http://commondatastorage.googleapis.com/books1000/'

def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [ ]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [ ]:
import random
def img_sample(img_folders, sample_size=1):
    for folder in img_folders:
        print(folder)
        imgs = random.sample(os.listdir(folder), sample_size)
        for img in imgs:
            img = os.path.join(folder, img)
            display(Image(img))

In [ ]:
img_sample(train_folders)

In [ ]:
img_sample(test_folders)

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [ ]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  for image_index, image in enumerate(image_files):
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file, flatten=True).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[image_index, :, :] = image_data
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  num_images = image_index + 1
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [ ]:
def pickle_sample(datasets, sample_size=1):
    for data_set in datasets:
        with open(data_set, 'rb') as f:
            imgs = random.sample(pickle.load(f), sample_size)
            for img in imgs:
                plt.imshow(img, cmap='Greys_r')
                plt.show()

In [ ]:
pickle_sample(train_datasets)

In [ ]:
pickle_sample(test_datasets)

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [ ]:
for i, data_set in enumerate(train_datasets):
    with open(data_set, 'rb') as f:
        size = pickle.load(f).shape[0]
        print("There are %d examples of %s in the training data" % (size, chr(i+65)))

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

Finally, let's save the data for later reuse:

In [ ]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [ ]:
def overlap_find(dataset1, dataset2):
    overlap = []
    for i, img1 in enumerate(dataset1):
        for j, img2 in enumerate(dataset2):     
            if np.array_equal(img1, img2):
                overlap.append(j)
                break
    return overlap

def near_overlap_find(dataset1, dataset2, max_norm=10):
    overlap = []
    dataset1_flat = dataset1.reshape(dataset1.shape[0], dataset1.shape[1]*dataset1.shape[2])
    dataset2_flat = dataset2.reshape(dataset2.shape[0], dataset2.shape[1]*dataset2.shape[2])
    for i, img1 in enumerate(dataset1_flat):
        norm = np.sum(np.abs(dataset2_flat - img1), axis=1)
        duplicates = np.where(norm < max_norm)
        if len(duplicates[0]):
            overlap.append(duplicates[0])
    return overlap

In [ ]:
%time overlap_test = overlap_find(test_dataset[:50], train_dataset)
print('Number of overlaps:', len(overlap_test))

In [ ]:
%time overlap_near_test = near_overlap_find(test_dataset[:50], train_dataset)
print('Number of overlaps:', len(overlap_near_test))

Let's try comparing hashes instead of using np.array_equal

In [ ]:
import hashlib
def overlap_find(dataset1, dataset2):
    hash1 = np.array([hashlib.sha256(img).hexdigest() for img in dataset1])
    hash2 = np.array([hashlib.sha256(img).hexdigest() for img in dataset2])
    overlap = []
    for i, hsh in enumerate(hash1):
        duplicates = np.where(hash2==hsh)
        if len(duplicates[0]):
            overlap.append(i)
    return overlap

In [ ]:
%time overlap_test = overlap_find(test_dataset[:200], train_dataset)
print('Number of overlaps:', len(overlap_test))

In [ ]:
def sanitize(dataset, train_set, labels):
    overlap = overlap_find(dataset, train_set)
    return np.delete(dataset, overlap, 0), np.delete(labels, overlap, 0)

In [ ]:
%time test_dataset_clean, test_labels_clean = sanitize(test_dataset, train_dataset, test_labels)
print('Number of overlaps in test dataset:', len(test_labels)-len(test_labels_clean))

In [ ]:
%time valid_dataset_clean, valid_labels_clean = sanitize(valid_dataset, train_dataset, valid_labels)
print('Number of overlaps in valid dataset:', len(valid_labels)-len(valid_labels_clean))

In [ ]:
pickle_file = 'notMNIST_clean.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset_clean,
    'valid_labels': valid_labels_clean,
    'test_dataset': test_dataset_clean,
    'test_labels': test_labels_clean,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [ ]:
Xtest = test_dataset_clean.reshape(test_dataset_clean.shape[0], test_dataset_clean.shape[1] 
                                   * test_dataset_clean.shape[2])
ytest = test_labels_clean

In [ ]:
from sklearn import linear_model
def train(train_dataset, train_labels, sample, **lgargs):
    Xtrain = train_dataset[:sample].reshape(sample, train_dataset.shape[1]*train_dataset.shape[2])
    ytrain = train_labels[:sample]
    logreg = linear_model.LogisticRegression(**lgargs)
    %time logreg.fit(Xtrain, ytrain)
    return logreg

In [ ]:
logreg = train(train_dataset, train_labels, 50)
logreg.score(Xtest, ytest)

In [ ]:
logreg = train(train_dataset, train_labels, 200)
logreg.score(Xtest, ytest)

In [ ]:
logreg = train(train_dataset, train_labels, 1000)
logreg.score(Xtest, ytest)

In [ ]:
logreg = train(train_dataset, train_labels, 5000, C=0.5)
logreg.score(Xtest, ytest)

In [ ]:
def disp_sample_dataset(dataset, labels, prob=None, sample=8, numrows=2):
    items = random.sample(range(len(labels)), sample)
    for i, item in enumerate(items):
        plt.subplot(numrows, sample//numrows, i+1)
        plt.axis('off')
        title = chr(labels[item]+65)
        if len(prob):
            title += "\n%.2f%%" % (prob[item]*100)
        plt.title(title, y=1.08)
        plt.imshow(dataset[item], cmap='Greys_r')

In [ ]:
pred_labels = logreg.predict(Xtest)
disp_sample_dataset(test_dataset_clean, pred_labels, sample=10)

Trying Tensorflow

In [ ]:
n = np.max(train_labels)+1
testcnt = len(ytest)
ytestenc = np.zeros((testcnt, n))
ytestenc[np.arange(testcnt), ytest] = 1

In [ ]:
import tensorflow as tf

def train_tensor(train_dataset, train_labels, sample):
    n = np.max(train_labels)+1
    img_size = train_dataset.shape[1]*train_dataset.shape[2]
    Xtrain = train_dataset[:sample].reshape(sample, img_size)
    ytrain = np.zeros((sample, n))
    ytrain[np.arange(sample), train_labels[:sample]] = 1
    
    W = tf.Variable(tf.zeros([img_size, n]))
    b = tf.Variable(tf.zeros([n]))
    x = tf.placeholder(tf.float32, [None, img_size])
    y = tf.placeholder(tf.float32, [None, n])
    pred = tf.nn.softmax(tf.matmul(x, W) + b)
    loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    for i in range(1000):
        perm = np.random.permutation(sample)
        Xtrain = Xtrain[perm, :]
        ytrain = ytrain[perm, :]
        sess.run(train_step, feed_dict={x: Xtrain[:100, :], y: ytrain[:100, :]})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(pred,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("%.2f%% accuracy" % (sess.run(accuracy, feed_dict={x: Xtest, y: ytestenc}) * 100))
    result = tf.argmax(pred,1)
    result = sess.run(result, feed_dict={x: Xtest})
    result = np.array(result, dtype=int)
    
    W = sess.run(W)
    b = sess.run(b)
    sess.close()
    return W, b, result

In [ ]:
W, b, pred = train_tensor(train_dataset, train_labels, 1000)

In [ ]:
W, b, pred = train_tensor(train_dataset, train_labels, len(train_labels))

In [ ]:
img_size = Xtest.shape[1]

x = tf.placeholder(tf.float32, [None, img_size])
perc = tf.nn.softmax(tf.matmul(x, W) + b)
pred = tf.argmax(perc, 1)
prob = tf.reduce_max(perc, 1)

feed_dict = {x: Xtest}
sess = tf.Session()
disp_sample_dataset(test_dataset_clean, sess.run(pred, feed_dict=feed_dict), prob=sess.run(prob, feed_dict=feed_dict), sample=8, numrows=2)
sess.close()

In [ ]:
img = load_letter("mypics", 1)
img_flat = img.reshape(img.shape[0], img.shape[1]*img.shape[2])
img_size = img_flat.shape[1]

x = tf.placeholder(tf.float32, [None, img_size])
perc = tf.nn.softmax(tf.matmul(x, W) + b)
pred = tf.argmax(perc, 1)
prob = tf.reduce_max(perc, 1)

feed_dict = {x: img_flat}
sess = tf.Session()
disp_sample_dataset(img, sess.run(pred, feed_dict=feed_dict), prob=sess.run(prob, feed_dict=feed_dict), sample=5, numrows=1)
sess.close()